We will base the model on https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

VGG16 finetuning

Really based on: 
https://flyyufelix.github.io/2016/10/08/fine-tuning-in-keras-part2.html

From https://github.com/flyyufelix/cnn_finetune/blob/master/vgg16.py We copy the script

In [5]:
import keras
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from sklearn.metrics import log_loss
from load_cifar10 import load_cifar10_data
import scipy
from keras.preprocessing.image import ImageDataGenerator

make sure the cifar 10 code is available for importing: 
https://github.com/Hvass-Labs/TensorFlow-Tutorials/issues/6
All dependencies are to be found at : https://github.com/flyyufelix/cnn_finetune
https://github.com/flyyufelix/cnn_finetune/blob/master/load_cifar10.py
install cv2: - https://stackoverflow.com/questions/23119413/how-do-i-install-python-opencv-through-conda 

Note: to take care of later:   ( inspiration: https://github.com/flyyufelix/cnn_finetune)
 For Tensorflow 
 Switch RGB to BGR order 
x = x[:, :, :, ::-1]  




# Define the model


In [15]:
def vgg16_model(img_rows, img_cols, channel=1, num_classes=None):
    """VGG 16 Model for Keras
    Model Schema is based on 
    https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3
    ImageNet Pretrained Weights 
    https://drive.google.com/file/d/0Bz7KyqmuGsilT0J5dmRCM0ROVHc/view?usp=sharing
    Parameters:
      img_rows, img_cols - resolution of inputs
      channel - 1 for grayscale, 3 for color 
      num_classes - number of categories for our classification task
    """
    model = Sequential()
    
    #conv 1
    model.add(Reshape((1,224,224)))
    model.add(ZeroPadding2D((1, 1), input_shape=(channel, img_rows, img_cols)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    #conv 2
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    #conv 3
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    #conv 4
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    #conv 5
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    #Fully connected
    # Add Fully Connected Layer
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    # Loads ImageNet pre-trained data
    model.load_weights('imagenet_models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

    # Truncate and replace softmax layer for transfer learning
    model.layers.pop()
    model.outputs = [model.layers[-1].output]
    model.layers[-1].outbound_nodes = []
    model.add(Dense(num_classes, activation='softmax'))

    # Uncomment below to set the first 10 layers to non-trainable (weights will not be updated)
    for layer in model.layers[:10]:
        layer.trainable = False

    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Start The training

In [16]:
# Example to fine-tune on 3000 samples from Cifar10

img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 6
batch_size = 16 
nb_epoch = 10

In [17]:
# Load Cifar10 data. Please implement your own load_data() module for your own dataset
#X_train, Y_train, X_valid, Y_valid = load_cifar10_data(img_rows, img_cols)

# Load our model
model = vgg16_model(img_rows, img_cols, channel, num_classes)


/home/jgrietens/anaconda3/envs/tf-keras/lib/python3.6/site-packages/ipykernel/__main__.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/home/jgrietens/anaconda3/envs/tf-keras/lib/python3.6/site-packages/ipykernel/__main__.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/home/jgrietens/anaconda3/envs/tf-keras/lib/python3.6/site-packages/ipykernel/__main__.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
/home/jgrietens/anaconda3/envs/tf-keras/lib/python3.6/site-packages/ipykernel/__main__.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
/home/jgrietens/anaconda3/envs/tf-keras/lib/python3.6/site-packages/ipykernel/__main__.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
/home/jgrietens/anaconda3/envs/t

ValueError: You are trying to load a weight file containing 13 layers into a model with 0 layers.

### Create our own load_data for our own dataset

In [ ]:
#flow_from_directory(directory)
#https://keras.io/preprocessing/image/

In [ ]:
#Create a Data Generator for data augmentation
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [ ]:
# First make saperate train and validation folders. 
train_generator = train_datagen.flow_from_directory(
        '/home/jgrietens/Documents/Load hackaton/1.0/data/dataset-resized-train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

In [ ]:
# Start Fine-tuning
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, shuffle=True, verbose=1, validation_data=(X_valid, Y_valid),
)


# Classify

Make sure to resize the images to (384, 512, 3). Some inspiration might be found here : https://github.com/garythung/trashnet/tree/master/data

also, look at resizing in load_cifar10.py:

https://github.com/flyyufelix/cnn_finetune/blob/master/load_cifar10.py

In [ ]:
#Find out the image size
im_path = '/home/jgrietens//Documents/Load hackaton/1.0/data/dataset-resized/cardboard/cardboard248.jpg'
img = scipy.misc.imread(im_path)
print('the image shape is : ' + str(img.shape))

In [ ]:
# Make predictions
predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)

# Cross-entropy loss score
score = log_loss(Y_valid, predictions_valid)

